# Library 불러오기

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import random
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv1D, BatchNormalization, Activation, Dropout
from tensorflow.keras import Model

# 태양광 발전량 데이터 불러오기와 학습 및 추론 데이터 나누기

In [2]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

In [3]:
energy = pd.read_csv('data/energy.csv')
energy['time'] = energy['time'].apply(lambda x:convert_time(x)) 
energy['time']=pd.to_datetime(energy['time']) + pd.DateOffset(hours=1)
energy['tmp time']=pd.to_datetime(energy['time']) - pd.DateOffset(hours=1) # 00:00:00 시간을 포함하기위하여
energy[['ulsan']]=energy[['ulsan']].astype(float)
ulsan_energy=energy[['time','ulsan','tmp time']]
train_energy=ulsan_energy[~(ulsan_energy['tmp time'].dt.year==2021)].reset_index(drop=True)
train_energy=train_energy[24::].reset_index(drop=True) # 2월 29일 예보 데이터가 없음
validation_energy=ulsan_energy[ulsan_energy['tmp time'].dt.year==2021].reset_index(drop=True)

# 일기 예보 데이터 불러오기와 학습 및 추론 데이터 나누기

예측일 전날 23시 예보 데이터를 이용하는 모델입니다.

In [4]:
def to_date(x):
    return pd.DateOffset(hours=x)

In [5]:
ulsan_fcst = pd.read_csv('data/ulsan_fcst_data.csv')
ulsan_fcst['Forecast time'] = pd.to_datetime(ulsan_fcst['Forecast time'])
ulsan_fcst['Predict time']=ulsan_fcst['Forecast time'] + ulsan_fcst['forecast'].map(to_date)
ulsan_fcst['tmp time']=ulsan_fcst['Predict time'] - pd.DateOffset(hours=1)  # 00:00:00 시간을 포함하기위하여
ulsan_fcst_sort=ulsan_fcst.sort_values(['Predict time', 'forecast'],ascending=[True, False])
ulsan_fcst_remove=ulsan_fcst_sort[ulsan_fcst_sort['Forecast time'].dt.hour==23].reset_index(drop=True) # 23시 데이터만 사용

D:\Anaconda\envs\data_engeneering\lib\site-packages\pandas\core\arrays\datetimelike.py:1111: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,


In [6]:
ulsan_fcst_remove=ulsan_fcst_remove[ulsan_fcst_remove['forecast']<=25] # 다음날 24시간을 다 포함하는 시간까지만 forecast 사용
ulsan_fcst_remove=ulsan_fcst_remove[~(ulsan_fcst_remove['Forecast time'].dt.day==ulsan_fcst_remove['Predict time'].dt.day)]
train_ulsan_fcst=ulsan_fcst_remove[~(ulsan_fcst_remove['tmp time'].dt.year==2021)].reset_index(drop=True)
validation_ulsan_fcst=ulsan_fcst_remove[(ulsan_fcst_remove['tmp time'].dt.year==2021) & (ulsan_fcst_remove['tmp time'].dt.month==1)].reset_index(drop=True)
test_ulsan_fcst=ulsan_fcst_remove[(ulsan_fcst_remove['tmp time'].dt.year==2021) & (ulsan_fcst_remove['tmp time'].dt.month==2)].reset_index(drop=True)

In [18]:
def data_gen(df, energy, target):
    df=df.reset_index(drop=True)
    df['Predict time']=df['Predict time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    df['date'] = df['Predict time'].str.split(' ').str[0]
    df['hour'] = df['Predict time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
    df['year'] = df['date'].str.split('-').str[0].astype(int)
    df['month'] = df['date'].str.split('-').str[1].astype(int)
    df['day'] = df['date'].str.split('-').str[2].astype(int)

    train_df=df[['month', 'hour', 'Temperature','Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]

    train_df['Humidity']=train_df['Humidity']/100
    train_df['WindDirection']=train_df['WindDirection']/360

    for idx in range(int(len(train_df)/8)):
        if idx == 0:
            train=np.array(train_df.iloc[idx*8:(idx+1)*8,:]).reshape(1,8,7)
        else:        
            train=np.concatenate( (train, np.array(train_df.iloc[idx*8:(idx+1)*8,:]).reshape(1,8,7)),axis=0)

    label=np.array(energy[target]).reshape(int(len(energy)/24),24)

    return train, label


In [8]:
train_x, train_y= data_gen(train_ulsan_fcst,train_energy, 'ulsan')
idx=(np.isnan(train_x).sum(axis=2).sum(axis=1))==0
train_x=train_x[idx]
train_y=train_y[idx]
shuffle_idx = np.arange(len(train_x))
np.random.shuffle(shuffle_idx)
train_x = train_x[shuffle_idx]
train_y = train_y[shuffle_idx]
print('Train set Input shape : ' + str(train_x.shape))
print('Train set Label shape : ' + str(train_y.shape))

validation_x, validation_y = data_gen(validation_ulsan_fcst,validation_energy)
print('Validation set Input shape : ' + str(validation_x.shape))
print('Validation set Label shape : ' + str(validation_y.shape))

test_x, _ = data_gen(test_ulsan_fcst,validation_energy) # test 데이터 생성
print('Test set Input shape : ' + str(test_x.shape))

Train set Input shape : (1036, 8, 7)
Train set Label shape : (1036, 24)
Validation set Input shape : (31, 8, 7)
Validation set Label shape : (31, 24)
Test set Input shape : (28, 8, 7)


D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Model - Dilated causal 1D CNN

In [9]:
def regression_dilated_cnn(Model_input):
    x = Conv1D(8, 3, padding='causal')(Model_input)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(16, 3, padding='causal', dilation_rate=2)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(32, 3, padding='causal', dilation_rate=4)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(64, 3, padding='causal', dilation_rate=8)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(64, 1)(x)
    x = Flatten()(x)
    x = Dense(50)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(0.2)(x)

    x = Dense(24)(x)

    output = Model(Model_input, x, name='regression_dilated_cnn')

    return output

# Train

In [10]:
model_inputs = keras.Input(shape=(8, 7))
AI_model = regression_dilated_cnn(model_inputs)
print(AI_model.summary())
AI_model.compile(loss='mse', optimizer='adam')
AI_model.fit(train_x, train_y, epochs=200, batch_size=32)  

Model: "regression_dilated_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 7)]            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 8, 8)              176       
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8)              32        
_________________________________________________________________
activation (Activation)      (None, 8, 8)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 8)              0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 16)             400       
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 16)  

33/33 [==============================] - 0s 4ms/step - loss: 1775.5225
Epoch 57/200
33/33 [==============================] - 0s 4ms/step - loss: 1677.1484
Epoch 58/200
33/33 [==============================] - 0s 4ms/step - loss: 1686.8179
Epoch 59/200
33/33 [==============================] - 0s 4ms/step - loss: 1706.9807
Epoch 60/200
33/33 [==============================] - 0s 4ms/step - loss: 1676.0769
Epoch 61/200
33/33 [==============================] - 0s 4ms/step - loss: 1702.8636
Epoch 62/200
33/33 [==============================] - 0s 4ms/step - loss: 1579.9583
Epoch 63/200
33/33 [==============================] - 0s 4ms/step - loss: 1598.4490
Epoch 64/200
33/33 [==============================] - 0s 4ms/step - loss: 1656.3656
Epoch 65/200
33/33 [==============================] - 0s 4ms/step - loss: 1679.5183
Epoch 66/200
33/33 [==============================] - 0s 4ms/step - loss: 1660.7285
Epoch 67/200
33/33 [==============================] - 0s 4ms/step - loss: 1604.9747
Epoch

33/33 [==============================] - 0s 4ms/step - loss: 1453.1494
Epoch 154/200
33/33 [==============================] - 0s 4ms/step - loss: 1358.1252
Epoch 155/200
33/33 [==============================] - 0s 4ms/step - loss: 1426.0092
Epoch 156/200
33/33 [==============================] - 0s 4ms/step - loss: 1396.3342
Epoch 157/200
33/33 [==============================] - 0s 4ms/step - loss: 1398.4778
Epoch 158/200
33/33 [==============================] - 0s 4ms/step - loss: 1358.6390
Epoch 159/200
33/33 [==============================] - 0s 4ms/step - loss: 1411.5227
Epoch 160/200
33/33 [==============================] - 0s 4ms/step - loss: 1442.1783
Epoch 161/200
33/33 [==============================] - 0s 4ms/step - loss: 1448.5615
Epoch 162/200
33/33 [==============================] - 0s 4ms/step - loss: 1366.0410
Epoch 163/200
33/33 [==============================] - 0s 4ms/step - loss: 1330.0743
Epoch 164/200
33/33 [==============================] - 0s 4ms/step - loss: 1459

# Test

In [11]:
prediction=AI_model.predict(validation_x)
prediction=np.abs(prediction)
prediction=np.round(prediction,0)
predict_y=np.ravel(prediction)
valid_y=np.ravel(validation_y)

## 울산 예측

In [12]:
pred_test = AI_model.predict(test_x) # test 데이터로 발전량 결과 예측

In [13]:
ulsan_pred = pred_test.reshape(672, 1)

## 수상 태양광 예측

In [59]:
floating_energy=energy[['time','dangjin_floating','tmp time']]
train_energy=floating_energy[~(floating_energy['tmp time'].dt.year==2021)].reset_index(drop=True)
train_energy=train_energy[24::].reset_index(drop=True) # 2월 29일 예보 데이터가 없음
validation_energy=floating_energy[floating_energy['tmp time'].dt.year==2021].reset_index(drop=True)

In [60]:
dangjin_fcst = pd.read_csv('data/dangjin_fcst_data.csv')
dangjin_fcst['Forecast time'] = pd.to_datetime(dangjin_fcst['Forecast time'])
dangjin_fcst['Predict time']=dangjin_fcst['Forecast time'] + dangjin_fcst['forecast'].map(to_date)
dangjin_fcst['tmp time']=dangjin_fcst['Predict time'] - pd.DateOffset(hours=1)  # 00:00:00 시간을 포함하기위하여
dangjin_fcst_sort=dangjin_fcst.sort_values(['Predict time', 'forecast'],ascending=[True, False])
dangjin_fcst_remove=dangjin_fcst_sort[dangjin_fcst_sort['Forecast time'].dt.hour==23].reset_index(drop=True) # 23시 데이터만 사용

D:\Anaconda\envs\data_engeneering\lib\site-packages\pandas\core\arrays\datetimelike.py:1111: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized
  PerformanceWarning,


In [61]:
dangjin_fcst_remove=dangjin_fcst_remove[dangjin_fcst_remove['forecast']<=25] # 다음날 24시간을 다 포함하는 시간까지만 forecast 사용
dangjin_fcst_remove=dangjin_fcst_remove[~(dangjin_fcst_remove['Forecast time'].dt.day==dangjin_fcst_remove['Predict time'].dt.day)]
train_dangjin_fcst=dangjin_fcst_remove[~(dangjin_fcst_remove['tmp time'].dt.year==2021)].reset_index(drop=True)
validation_dangjin_fcst=dangjin_fcst_remove[(dangjin_fcst_remove['tmp time'].dt.year==2021) & (dangjin_fcst_remove['tmp time'].dt.month==1)].reset_index(drop=True)
test_dangjin_fcst=dangjin_fcst_remove[(dangjin_fcst_remove['tmp time'].dt.year==2021) & (dangjin_fcst_remove['tmp time'].dt.month==2)].reset_index(drop=True)

In [62]:
train_x, train_y= data_gen(train_dangjin_fcst,train_energy, 'dangjin_floating')
idx=(np.isnan(train_x).sum(axis=2).sum(axis=1))==0
train_x=train_x[idx]
train_y=train_y[idx]
shuffle_idx = np.arange(len(train_x))
np.random.shuffle(shuffle_idx)
train_x = train_x[shuffle_idx]
train_y = train_y[shuffle_idx]
print('Train set Input shape : ' + str(train_x.shape))
print('Train set Label shape : ' + str(train_y.shape))

validation_x, validation_y = data_gen(validation_dangjin_fcst,validation_energy, 'dangjin_floating')
print('Validation set Input shape : ' + str(validation_x.shape))
print('Validation set Label shape : ' + str(validation_y.shape))

test_x, _ = data_gen(test_dangjin_fcst,validation_energy, 'dangjin_floating') # test 데이터 생성
print('Test set Input shape : ' + str(test_x.shape))

Train set Input shape : (1036, 8, 7)
Train set Label shape : (1036, 24)
Validation set Input shape : (31, 8, 7)
Validation set Label shape : (31, 24)
Test set Input shape : (28, 8, 7)


D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
floating_energy[:24]

,time,dangjin_floating,tmp time
0,2018-03-01 01:00:00,0.0,2018-03-01 00:00:00
1,2018-03-01 02:00:00,0.0,2018-03-01 01:00:00
2,2018-03-01 03:00:00,0.0,2018-03-01 02:00:00
3,2018-03-01 04:00:00,0.0,2018-03-01 03:00:00
4,2018-03-01 05:00:00,0.0,2018-03-01 04:00:00
5,2018-03-01 06:00:00,0.0,2018-03-01 05:00:00
6,2018-03-01 07:00:00,0.0,2018-03-01 06:00:00
7,2018-03-01 08:00:00,0.0,2018-03-01 07:00:00
8,2018-03-01 09:00:00,36.0,2018-03-01 08:00:00
9,2018-03-01 10:00:00,313.0,2018-03-01 09:00:00


In [68]:
dangjin_energy[:24]

,time,dangjin,tmp time
0,2018-03-01 01:00:00,0,2018-03-01 00:00:00
1,2018-03-01 02:00:00,0,2018-03-01 01:00:00
2,2018-03-01 03:00:00,0,2018-03-01 02:00:00
3,2018-03-01 04:00:00,0,2018-03-01 03:00:00
4,2018-03-01 05:00:00,0,2018-03-01 04:00:00
5,2018-03-01 06:00:00,0,2018-03-01 05:00:00
6,2018-03-01 07:00:00,0,2018-03-01 06:00:00
7,2018-03-01 08:00:00,0,2018-03-01 07:00:00
8,2018-03-01 09:00:00,37,2018-03-01 08:00:00
9,2018-03-01 10:00:00,318,2018-03-01 09:00:00


In [64]:
test_x

array([[[ 2.        ,  3.        ,  7.        , ...,  3.4       ,
          0.575     ,  4.        ],
        [ 2.        ,  6.        ,  6.        , ...,  2.5       ,
          0.61666667,  4.        ],
        [ 2.        ,  9.        ,  6.        , ...,  3.3       ,
          0.64444444,  4.        ],
        ...,
        [ 2.        , 18.        ,  4.        , ...,  3.6       ,
          0.85833333,  3.        ],
        [ 2.        , 21.        ,  1.        , ...,  6.6       ,
          0.90277778,  3.        ],
        [ 2.        ,  0.        , -3.        , ...,  7.4       ,
          0.92222222,  1.        ]],

       [[ 2.        ,  3.        , -5.        , ...,  7.6       ,
          0.90833333,  1.        ],
        [ 2.        ,  6.        , -6.        , ...,  7.1       ,
          0.90555556,  1.        ],
        [ 2.        ,  9.        , -6.        , ...,  7.        ,
          0.90277778,  1.        ],
        ...,
        [ 2.        , 18.        , -4.        , ...,  

In [69]:
model_inputs = keras.Input(shape=(8, 7))
AI_model = regression_dilated_cnn(model_inputs)
print(AI_model.summary())
AI_model.compile(loss='mse', optimizer='adam')
AI_model.fit(train_x, train_y, epochs=200, batch_size=32)  

Model: "regression_dilated_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 8, 7)]            0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 8, 8)              176       
_________________________________________________________________
batch_normalization_35 (Batc (None, 8, 8)              32        
_________________________________________________________________
activation_35 (Activation)   (None, 8, 8)              0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 8, 8)              0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 8, 16)             400       
_________________________________________________________________
batch_normalization_36 (Batc (None, 8, 16)  

33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 61/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 62/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 63/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 64/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 65/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 66/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 67/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 68/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 69/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 70/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 71/200
33/33 [==============================] - 0s 3ms/step - loss: nan
Epoch 72/200
33/33 [==============================] - 0s 4ms/step - loss: nan

33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 165/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 166/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 167/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 168/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 169/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 170/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 171/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 172/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 173/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 174/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 175/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 176/200
33/33 [==============================] - 0s 4ms/step

In [24]:
pred_test = AI_model.predict(test_x) # test 데이터로 발전량 결과 예측

In [25]:
dangjin_floating_pred = pred_test.reshape(672, 1)

## 자재창고 태양광 예측

In [26]:
warehouse_energy=energy[['time','dangjin_warehouse','tmp time']]
train_energy=warehouse_energy[~(warehouse_energy['tmp time'].dt.year==2021)].reset_index(drop=True)
train_energy=train_energy[24::].reset_index(drop=True) # 2월 29일 예보 데이터가 없음
validation_energy=warehouse_energy[warehouse_energy['tmp time'].dt.year==2021].reset_index(drop=True)

In [27]:
train_x, train_y= data_gen(train_dangjin_fcst,train_energy, 'dangjin_warehouse')
idx=(np.isnan(train_x).sum(axis=2).sum(axis=1))==0
train_x=train_x[idx]
train_y=train_y[idx]
shuffle_idx = np.arange(len(train_x))
np.random.shuffle(shuffle_idx)
train_x = train_x[shuffle_idx]
train_y = train_y[shuffle_idx]
print('Train set Input shape : ' + str(train_x.shape))
print('Train set Label shape : ' + str(train_y.shape))

validation_x, validation_y = data_gen(validation_dangjin_fcst,validation_energy, 'dangjin_warehouse')
print('Validation set Input shape : ' + str(validation_x.shape))
print('Validation set Label shape : ' + str(validation_y.shape))

test_x, _ = data_gen(test_dangjin_fcst,validation_energy, 'dangjin_warehouse') # test 데이터 생성
print('Test set Input shape : ' + str(test_x.shape))

Train set Input shape : (1036, 8, 7)
Train set Label shape : (1036, 24)
Validation set Input shape : (31, 8, 7)
Validation set Label shape : (31, 24)
Test set Input shape : (28, 8, 7)


D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
model_inputs = keras.Input(shape=(8, 7))
AI_model = regression_dilated_cnn(model_inputs)
print(AI_model.summary())
AI_model.compile(loss='mse', optimizer='adam')
AI_model.fit(train_x, train_y, epochs=200, batch_size=32)  

Model: "regression_dilated_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8, 7)]            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 8, 8)              176       
_________________________________________________________________
batch_normalization_10 (Batc (None, 8, 8)              32        
_________________________________________________________________
activation_10 (Activation)   (None, 8, 8)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8)              0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 8, 16)             400       
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 16)  

33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 61/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 62/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 63/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 64/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 65/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 66/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 67/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 68/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 69/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 70/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 71/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 72/200
33/33 [==============================] - 0s 4ms/step - loss: nan

33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 165/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 166/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 167/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 168/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 169/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 170/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 171/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 172/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 173/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 174/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 175/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 176/200
33/33 [==============================] - 0s 4ms/step

In [29]:
pred_test = AI_model.predict(test_x) # test 데이터로 발전량 결과 예측

In [30]:
dangjin_warehouse_pred = pred_test.reshape(672, 1)

## 당진 태양광 예측

In [54]:
dangjin_energy=energy[['time','dangjin','tmp time']]
train_energy=dangjin_energy[~(dangjin_energy['tmp time'].dt.year==2021)].reset_index(drop=True)
train_energy=train_energy[24::].reset_index(drop=True) # 2월 29일 예보 데이터가 없음
validation_energy=dangjin_energy[dangjin_energy['tmp time'].dt.year==2021].reset_index(drop=True)

In [55]:
train_x, train_y= data_gen(train_dangjin_fcst,train_energy, 'dangjin')
idx=(np.isnan(train_x).sum(axis=2).sum(axis=1))==0
train_x=train_x[idx]
train_y=train_y[idx]
shuffle_idx = np.arange(len(train_x))
np.random.shuffle(shuffle_idx)
train_x = train_x[shuffle_idx]
train_y = train_y[shuffle_idx]
print('Train set Input shape : ' + str(train_x.shape))
print('Train set Label shape : ' + str(train_y.shape))

validation_x, validation_y = data_gen(validation_dangjin_fcst,validation_energy, 'dangjin')
print('Validation set Input shape : ' + str(validation_x.shape))
print('Validation set Label shape : ' + str(validation_y.shape))

test_x, _ = data_gen(test_dangjin_fcst,validation_energy, 'dangjin') # test 데이터 생성
print('Test set Input shape : ' + str(test_x.shape))

Train set Input shape : (1036, 8, 7)
Train set Label shape : (1036, 24)
Validation set Input shape : (31, 8, 7)
Validation set Label shape : (31, 24)
Test set Input shape : (28, 8, 7)


D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
train_x

array([[[8.00000000e+00, 3.00000000e+00, 2.50000000e+01, ...,
         1.50000000e+00, 4.77777778e-01, 3.00000000e+00],
        [8.00000000e+00, 6.00000000e+00, 2.40000000e+01, ...,
         1.50000000e+00, 4.44444444e-01, 3.00000000e+00],
        [8.00000000e+00, 9.00000000e+00, 2.70000000e+01, ...,
         9.00000000e-01, 4.63888889e-01, 1.00000000e+00],
        ...,
        [8.00000000e+00, 1.80000000e+01, 2.70000000e+01, ...,
         2.50000000e+00, 8.83333333e-01, 1.00000000e+00],
        [8.00000000e+00, 2.10000000e+01, 2.50000000e+01, ...,
         2.20000000e+00, 9.00000000e-01, 1.00000000e+00],
        [8.00000000e+00, 0.00000000e+00, 2.40000000e+01, ...,
         2.20000000e+00, 9.55555556e-01, 1.00000000e+00]],

       [[1.10000000e+01, 3.00000000e+00, 7.00000000e+00, ...,
         6.20000000e+00, 9.58333333e-01, 1.00000000e+00],
        [1.10000000e+01, 6.00000000e+00, 7.00000000e+00, ...,
         6.00000000e+00, 9.88888889e-01, 1.00000000e+00],
        [1.10000000e+01, 

In [58]:
test_x

array([[[ 2.        ,  3.        ,  7.        , ...,  3.4       ,
          0.575     ,  4.        ],
        [ 2.        ,  6.        ,  6.        , ...,  2.5       ,
          0.61666667,  4.        ],
        [ 2.        ,  9.        ,  6.        , ...,  3.3       ,
          0.64444444,  4.        ],
        ...,
        [ 2.        , 18.        ,  4.        , ...,  3.6       ,
          0.85833333,  3.        ],
        [ 2.        , 21.        ,  1.        , ...,  6.6       ,
          0.90277778,  3.        ],
        [ 2.        ,  0.        , -3.        , ...,  7.4       ,
          0.92222222,  1.        ]],

       [[ 2.        ,  3.        , -5.        , ...,  7.6       ,
          0.90833333,  1.        ],
        [ 2.        ,  6.        , -6.        , ...,  7.1       ,
          0.90555556,  1.        ],
        [ 2.        ,  9.        , -6.        , ...,  7.        ,
          0.90277778,  1.        ],
        ...,
        [ 2.        , 18.        , -4.        , ...,  

In [70]:
model_inputs = keras.Input(shape=(8, 7))
AI_model = regression_dilated_cnn(model_inputs)
print(AI_model.summary())
AI_model.compile(loss='mse', optimizer='adam')
AI_model.fit(train_x, train_y, epochs=200, batch_size=32)  

Model: "regression_dilated_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 8, 7)]            0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 8, 8)              176       
_________________________________________________________________
batch_normalization_40 (Batc (None, 8, 8)              32        
_________________________________________________________________
activation_40 (Activation)   (None, 8, 8)              0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 8, 8)              0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 8, 16)             400       
_________________________________________________________________
batch_normalization_41 (Batc (None, 8, 16)  

33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 61/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 62/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 63/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 64/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 65/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 66/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 67/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 68/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 69/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 70/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 71/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 72/200
33/33 [==============================] - 0s 4ms/step - loss: nan

33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 165/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 166/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 167/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 168/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 169/200
33/33 [==============================] - 0s 4ms/step - loss: nan
Epoch 170/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 171/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 172/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 173/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 174/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 175/200
33/33 [==============================] - 0s 5ms/step - loss: nan
Epoch 176/200
33/33 [==============================] - 0s 5ms/step

In [34]:
pred_test = AI_model.predict(test_x) # test 데이터로 발전량 결과 예측

In [35]:
dangjin_pred = pred_test.reshape(672, 1)

In [46]:
dangjin_floating_pred

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      

In [38]:
submission = pd.read_csv('data/sample_submission.csv')

In [39]:
submission.iloc[:24*28, 1] = dangjin_floating_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [40]:
submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,NaN,NaN,0.000017,0.000051
1,2021-02-01 02:00:00,NaN,NaN,-0.000436,0.000102
2,2021-02-01 03:00:00,NaN,NaN,-0.000005,0.004581
3,2021-02-01 04:00:00,NaN,NaN,0.000700,0.000029
4,2021-02-01 05:00:00,NaN,NaN,0.005507,-0.001729
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,0.0,0.000000,0.000000
1388,2021-07-08 21:00:00,0.0,0.0,0.000000,0.000000
1389,2021-07-08 22:00:00,0.0,0.0,0.000000,0.000000
1390,2021-07-08 23:00:00,0.0,0.0,0.000000,0.000000
